# Create Embeddings

create embeddings based from data generated

In [20]:
import lancedb
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

df = pd.read_csv("../data/chatgpt_house_match_data.csv")
df.head()

,name,year,location,layout,price,description
0,Modern and Spacious House in Kawasaki,2015,"Kawasaki City, Takatsu Ward",3LDK,"¥200,000/month",This beautiful house was built in 2015 and is ...
1,Cozy House in Fujisawa,1995,"Fujisawa, Kanagawa Prefecture",2LDK,"¥150,000/month",This lovely two-bedroom house is located in th...
2,Saitama Green House,2014,"Saitama City, Omiya Ward",1LDK,"¥120,000/month",This cozy house is located in the quiet reside...
3,Spacious Yokohama House,2010,"Yokohama, Kanagawa",3LDK,"¥200,000/month",This modern house is located in a quiet neighb...
4,Modern House in Saitama,2018,"Saitama City, Omiya Ward",2LDK,"¥170,000/month",This modern house is perfect for individuals o...


In [35]:
import dotenv
from typing import List
from langchain_openai.embeddings import OpenAIEmbeddings
dotenv.load_dotenv()

model = OpenAIEmbeddings()

def embed(texts: List[str]):
    # 
    if isinstance(texts, str):
        texts = [texts]
    
    result = model.embed_documents(texts)
    return result

output = embed(["hello", "world"])
assert len(output) == 2, "Not Prepared For Batch Processing."
    

In [37]:
import lancedb
from lancedb.embeddings import with_embeddings

data = with_embeddings(embed, df, column="description", show_progress=True)
data.to_pandas().head(1)

100%|██████████| 1/1 [00:02<00:00,  2.60s/it]


,name,year,location,layout,price,description,vector
0,Modern and Spacious House in Kawasaki,2015,"Kawasaki City, Takatsu Ward",3LDK,"¥200,000/month",This beautiful house was built in 2015 and is ...,"[0.0111366855, 0.020672187, -0.022481656, 0.00..."


In [ ]:
!rm -rf ../tmp/lancedb

db = lancedb.connect("../tmp/lancedb")
tbl = db.create_table("house_match", data)
